In [3]:
import numpy as np
import pandas as pd
from collections import Counter
import math

In [4]:
# features
outlook = ['sunny', 'sunny', 'overcast', 'rain', 'rain', 'rain', 'overcast', 'sunny', 'sunny', 'rain',
           'sunny', 'overcast', 'overcast', 'rain']
temperature = ['hot', 'hot', 'hot', 'mild', 'cool', 'cool', 'cool', 'mild', 'cool', 'mild',
               'mild', 'mild', 'hot', 'mild']
humidity = ['high', 'high', 'high', 'high', 'normal', 'normal', 'normal', 'high', 'normal', 'normal',
           'normal', 'high', 'normal', 'high']
wind = ['weak', 'strong', 'weak', 'weak', 'weak', 'strong', 'strong', 'weak', 'weak', 'weak', 'strong',
       'strong', 'weak', 'strong']
play = ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']

In [5]:
print(len(outlook))
print(len(temperature))
print(len(humidity))
print(len(wind))
print(len(play))

14
14
14
14
14


In [6]:
data = pd.DataFrame({'outlook': outlook, 'temperature': temperature, 'humidity': humidity,
                    'wind': wind, 'play': play})
data

,outlook,temperature,humidity,wind,play
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes
5,rain,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rain,mild,normal,weak,yes


In [13]:
yes_count = data['play'].value_counts()['yes']
no_count = data['play'].value_counts()['no']

In [13]:
Counter(data['play']).values()

dict_values([5, 9])

In [12]:
dict(data['play'].value_counts())

{'yes': 9, 'no': 5}

In [16]:
# utilities
def unique_values(target):
    record = dict(target.value_counts())
    return record

def entropy(target):
    target_record = unique_values(target)
    total_instance = len(target)
    entropy_value = 0.0
    for i in target_record.values():
        entropy_value -= (i/total_instance) * math.log2(i/total_instance)
    return entropy_value

def info_gain(features, target, curr_feature):
    # entropy of the whole dataset
    dataset_entropy = entropy(target)
    feature_value = set(data[curr_feature])
    total_instance = len(target)
    curr_feature_entropy = 0.0
    for value in feature_value:
        value_position = data[curr_feature] == value
        value_entropy = entropy(pd.Series(target[value_position]))
        curr_feature_entropy += value_entropy
        
    information_gain = dataset_entropy - curr_feature_entropy
    return information_gain

In [52]:
print(entropy(data['play']))

0.9402859586706311


In [53]:
type(data['play'])

pandas.core.series.Series

In [47]:
t = unique_values(data['play'])
total_instance = len(data['play'])
entropy_value = 0.0
for i in t.values():
    entropy_value -= (i/total_instance) * math.log2(i/total_instance)
print(entropy_value)

0.9402859586706311


In [49]:
print(entropy(data['play']))

TypeError: entropy() takes 1 positional argument but 2 were given

In [25]:
subset_indices = data['outlook'] == "sunny"
subset_indices

0      True
1      True
2     False
3     False
4     False
5     False
6     False
7      True
8      True
9     False
10     True
11    False
12    False
13    False
Name: outlook, dtype: bool

In [36]:
data['play'][subset_indices]

0      no
1      no
7      no
8     yes
10    yes
Name: play, dtype: object

In [20]:
sum(subset_indices)

5

In [27]:
-(2/5 * math.log2(2/5) + 3/5 * math.log2(3/5))

0.9709505944546686

In [54]:
t = pd.Series(data['play'][subset_indices])